In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [3]:
from sklearn.preprocessing import LabelEncoder
lb=LabelEncoder()

In [4]:
df_org=pd.read_csv('train_jqd04QH.csv')
df_test=pd.read_csv('test_KaymcHn.csv')
df_sub=pd.read_csv('sample_submission_sxfcbdx.csv')
df_copy=df_org.copy()

In [5]:
def feature_preprocessing(data):
    x1=np.nan
    data['nan_main']=data.apply(lambda x:list(x.loc[['education_level','major_discipline','experience','company_size','company_type']].values).count(x1),axis=1)
    x=pd.DataFrame(data.city.value_counts(normalize=True))
    x['city_name']=x.index
    x.reset_index(drop=True,inplace=True)
    x.columns=['count','city']
    data=pd.merge(data,x,on='city',how='left')
    
    x=pd.DataFrame(data.city_development_index.value_counts(normalize=True))
    x['city_develo_rank']=x.index
    x.reset_index(drop=True,inplace=True)
    x.columns=['city_develo_rank', 'city_development_index']
    data=pd.merge(left=data,right=x,on='city_development_index',how='left')
    data
    
    return data


def feature_engineering(data):
    data['gender']=data['gender'].fillna(data['gender'].mode()[0])
    data['enrolled_university']=data['enrolled_university'].fillna(data['enrolled_university'].mode()[0])
    data['education_level']=data['education_level'].fillna(data['education_level'].mode()[0])
    data['major_discipline']=data['major_discipline'].fillna(data['major_discipline'].mode()[0])
    data['experience']=data['experience'].fillna(data['experience'].mode()[0])
    data['company_size']=data['company_size'].fillna(data['company_size'].mode()[0])
    data['company_type']=data['company_type'].fillna(data['company_type'].mode()[0])
    data['last_new_job']=data['last_new_job'].fillna(data['last_new_job'].mode()[0])
    return data

In [6]:
df_copy=feature_preprocessing(df_copy)
df_test=feature_preprocessing(df_test)
df_copy=feature_engineering(df_copy)
df_test=feature_engineering(df_test)

In [7]:
cols=df_copy.select_dtypes('object').columns
for i in cols:
    lb=LabelEncoder()
    unique_val=list(set(df_copy[i].unique().tolist()+df_test[i].unique().tolist()))
    lb.fit(unique_val)
    df_copy[i]=lb.transform(df_copy[i])
    df_test[i]=lb.transform(df_test[i])

In [8]:
df_copy

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,nan_main,count,city_develo_rank
0,23798,41,0.689,1,0,2,0,5,13,1,5,0,106,0,0,0.005719,0.005719
1,29166,112,0.923,1,0,2,0,5,5,7,1,0,69,0,0,0.007735,0.007735
2,46,48,0.910,1,0,2,0,5,16,4,4,1,4,0,0,0.090092,0.090092
3,18527,93,0.666,1,0,2,0,5,5,4,5,0,26,0,0,0.006482,0.006482
4,21751,2,0.887,1,1,2,2,5,18,4,5,1,88,1,2,0.014761,0.014761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,25366,5,0.920,1,0,0,0,5,15,7,5,0,71,0,0,0.237377,0.282423
18355,25545,49,0.920,1,1,2,0,2,6,4,5,0,160,0,0,0.045046,0.282423
18356,11514,13,0.926,1,0,2,2,5,2,4,5,2,18,0,0,0.080179,0.080179
18357,1689,103,0.939,1,0,2,0,5,21,0,5,2,41,0,0,0.018084,0.029631


In [51]:
df_org[df_org['target']==1]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1
6,11746,city_21,0.624,Male,Has relevent experience,no_enrollment,Graduate,STEM,6,10000+,Pvt Ltd,3,23,1
24,3472,city_16,0.910,Male,Has relevent experience,no_enrollment,High School,NaN,>20,500-999,Pvt Ltd,>4,54,1
38,32506,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,NaN,NaN,1,68,1
40,5104,city_75,0.939,Male,Has relevent experience,no_enrollment,Masters,STEM,7,NaN,NaN,1,16,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18315,30649,city_70,0.698,Female,Has relevent experience,Full time course,Graduate,STEM,9,50-99,Pvt Ltd,1,95,1
18326,15199,city_103,0.920,NaN,Has relevent experience,Part time course,Graduate,Business Degree,14,<10,Pvt Ltd,1,9,1
18329,22908,city_136,0.897,Male,Has relevent experience,no_enrollment,Masters,STEM,19,10/49,Pvt Ltd,1,16,1
18342,12930,city_73,0.754,NaN,Has relevent experience,no_enrollment,Graduate,STEM,17,50-99,Pvt Ltd,4,83,1


In [52]:
df_org[df_org['target']==0]

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
5,13342,city_21,0.624,Female,Has relevent experience,no_enrollment,Graduate,Other,8,5000-9999,Pvt Ltd,2,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18354,25366,city_103,0.920,Male,Has relevent experience,Full time course,Graduate,STEM,5,<10,Pvt Ltd,1,71,0
18355,25545,city_160,0.920,Male,No relevent experience,no_enrollment,Graduate,Humanities,15,50-99,Pvt Ltd,1,160,0
18356,11514,city_114,0.926,Male,Has relevent experience,no_enrollment,Masters,STEM,11,50-99,Pvt Ltd,3,18,0
18357,1689,city_75,0.939,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10/49,Pvt Ltd,3,41,0


In [48]:
x1=x[x['target']==0]

In [49]:
x1

,relevent_experience,training_hours,target
0,0,106,0
1,0,69,0
2,0,4,0
3,0,26,0
5,0,34,0
...,...,...,...
18354,0,71,0
18355,1,160,0
18356,0,18,0
18357,0,41,0


### Model Selection

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()

In [10]:
from xgboost import XGBClassifier
gbc=XGBClassifier()

In [13]:
from sklearn.model_selection import cross_val_score
import lightgbm as lgb

X=df_copy.drop(columns=["target","enrollee_id"])
y=df_copy["target"]

results=cross_val_score(gbc,X,y,cv=3,scoring="roc_auc",n_jobs=-1)
results

results.mean()

0.657305158157149

In [14]:
gbc.fit(X,y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [15]:
df_test.drop(['enrollee_id'],axis=1,inplace=True)

In [16]:
pred=gbc.predict_proba(df_test)[:,-1]

In [17]:
df_sub['target']=pred

In [18]:
df_sub.to_csv('output_xgboost_pl_without.csv',index=False)

In [19]:
s=pd.DataFrame(X.columns,gbc.feature_importances_)

In [21]:
s.sort_index(ascending=False)

,0
0.336792,city_development_index
0.187094,nan_main
0.069999,city_develo_rank
0.069114,training_hours
0.056971,last_new_job
0.048324,experience
0.044636,city
0.041636,enrolled_university
0.035799,count
0.031074,relevent_experience


In [22]:
import xgboost as xgb
from bayes_opt import BayesianOptimization
dtrain=xgb.DMatrix(X,y.ravel())
def bo_tune_xgb(max_depth,min_child_weight,colsample_bytree,subsample):
    params = {'max_depth': int(round(max_depth)),
              "colsample_bytree":round(colsample_bytree,1),
              'min_child_weight': int(round(min_child_weight)),
              'learning_rate':.1,
              'subsample':round(subsample,1),
              "boosting_type":"gbtree"
             }
    cv_result = xgb.cv(
        params,
        dtrain,
        num_boost_round=999,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    return cv_result['test-auc-mean'].max()



In [23]:
xgb_bo = BayesianOptimization(bo_tune_xgb, {'max_depth': (5,50), 
                                            'min_child_weight':(40,80),
                                            "colsample_bytree":(.5,1),
                                            "subsample":(.5,1)
                                            })

xgb_bo.maximize(init_points=5, n_iter=25)


xgb_params = xgb_bo.max['params']

print(xgb_params)

#Conversting the max_depth and n_estimator values from float to int
xgb_params['max_depth']= int(round(xgb_params['max_depth']))
xgb_params['min_child_weight']= int(round(xgb_params['min_child_weight']))
xgb_params['colsample_bytree']= round(xgb_params['colsample_bytree'],1)
#xgb_params['colsample_bytree']= .5
xgb_params['subsample']= round(xgb_params['subsample'])
print(xgb_params)

|   iter    |  target   | colsam... | max_depth | min_ch... | subsample |
-------------------------------------------------------------------------
|  1        |  0.6482   |  0.8467   |  43.31    |  47.45    |  0.7879   |
|  2        |  0.662    |  0.5273   |  5.42     |  43.37    |  0.9599   |
|  3        |  0.6532   |  0.6672   |  49.35    |  67.92    |  0.6517   |
|  4        |  0.6407   |  0.5761   |  40.65    |  41.03    |  0.7966   |
|  5        |  0.6519   |  0.8429   |  13.59    |  52.67    |  0.7477   |
|  6        |  0.6627   |  0.7832   |  5.023    |  79.27    |  0.6367   |
|  7        |  0.6662   |  0.8301   |  5.041    |  60.35    |  0.9639   |
|  8        |  0.6619   |  0.7139   |  6.728    |  64.01    |  0.8627   |
|  9        |  0.6615   |  0.9297   |  5.276    |  55.1     |  0.5339   |
|  10       |  0.6643   |  0.9833   |  5.088    |  75.07    |  0.9886   |
|  11       |  0.6626   |  0.5194   |  5.172    |  67.76    |  0.963    |
|  12       |  0.6653   |  0.9621   | 

In [24]:
xgbias=XGBClassifier(**xgb_params)

In [25]:
xgbias.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=60, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)

In [26]:
pred=xgbias.predict_proba(df_test)[:,-1]

In [27]:
df_sub['target']=pred

df_sub.to_csv('output_xgboost_bayesian.csv',index=False)

In [28]:
xgb_params

{'colsample_bytree': 0.8,
 'max_depth': 5,
 'min_child_weight': 60,
 'subsample': 1.0}

In [29]:
x=pd.DataFrame(X.columns,gbc.feature_importances_)

In [30]:
x.sort_index(ascending=False)

,0
0.336792,city_development_index
0.187094,nan_main
0.069999,city_develo_rank
0.069114,training_hours
0.056971,last_new_job
0.048324,experience
0.044636,city
0.041636,enrolled_university
0.035799,count
0.031074,relevent_experience
